In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#purpose: create our dataframe

#default column headers
col_headers = ['Name', 'Department', 'Title', 'Regular', 'Retro', 'Other', 'Overtime', 'Injured', 'Detail', 'Quinn', 'Total', 'Postal']

#create an empty df with default values
earnings_df = pd.DataFrame(columns = col_headers)

#add column 'Year' and set to NaN
earnings_df['Year'] = np.nan

for year in range(2011, 2020):
    
    #read in the CSV for the given year, set it to variable next_df
    next_df = pd.read_csv('./data/salaries_' + str(year) + '.csv', skiprows=1, names=col_headers, encoding='latin1')
    
    #add the column 'Year' to next_df and set to the given year
    next_df['Year'] = year
    
    #in the 2013 and 2014 datasets, title and department columns are in the wrong order
    if year == 2013 or year == 2014:
        
        #get the list of columns (including year)
        col_list = list(next_df)
        
        #swap the order of title and department in the list
        col_list[1], col_list[2] = col_list[2], col_list[1]
        
        #set the dataframe's columns to the new list
        next_df.columns = col_list
    
    #add next_df to earnings_df
    earnings_df = pd.concat([earnings_df, next_df], sort=False)

In [3]:
earnings_df.loc[earnings_df['Title'].str.match('Spec'), ['Name', 'Title', 'Postal']]

,Name,Title,Postal
3,"Ackerly,Lyn E.",Spec Library Asst I,02118-0000
85,"Apromollo,Pamela C.",Spec Library Asst I,02368-2727
121,"Baker,Alphonso",Spec Hvy Meo,02131-3835
196,"Biancamano,Nancy",Spec Library Asst I,02130-3700
230,"Bognanno,Lorna",Spec Asst II,02132-3143
...,...,...,...
22755,"Lindsey,Marie Irvy G.",Spec Library Asst V (BPL),4348
22758,"Apromollo,Pamela C.",Spec Library Asst II,2368
22793,"Davidson,Catherine A.",Spec Library Asst I (Branch),2129
22875,"Bradshaw,Adrienne R",Specialist (BTU),2301


In [4]:
#purpose: clean our dataframe

#targeted (hard-coded) cleaning for specific rows
earnings_df = earnings_df.loc[earnings_df['Department']!='DEPARTMENT_NAME']
earnings_df.loc[earnings_df['Department'] == 'Boston Cntr-Youth & Families', 'Department'] = 'Boston Cntr - Youth & Families'
earnings_df.loc[earnings_df['Department'] == 'DND Neighborhood Development', 'Department'] = 'Neighborhood Development'
earnings_df.loc[earnings_df['Name'].str.match('Ostiguy,David M'), 'Postal'] = '02327'
earnings_df.loc[earnings_df['Name'].str.match('Karales,George Alfred'), 'Postal'] = '02170'
earnings_df.loc[earnings_df['Name'].str.match('Smith,Kenneth J'), 'Postal'] = '02124'
earnings_df.loc[earnings_df['Name'].str.match('Thomas,Sarita J'), 'Postal'] = '02125'
earnings_df.loc[earnings_df['Name'].str.match('Morris,Judith A.'), 'Postal'] = '02170'
earnings_df.loc[earnings_df['Name'].str.match('Mendez,Jose R'), 'Postal'] = '02135'
earnings_df.loc[earnings_df['Name'].str.match('Morrison,June'), 'Postal'] = '02481'

#dimensions are qualitative columns, facts are quantitative columns
facts = ['Regular', 'Retro', 'Other', 'Overtime', 'Injured', 'Detail', 'Quinn', 'Total']
    
##cast year to type 'int'
earnings_df['Year'] = earnings_df['Year'].astype(int)

#clean the facts columns and convert from type 'object' to 'float'
earnings_df[facts] = earnings_df[facts].astype(str).applymap(lambda x: x.strip())
earnings_df[facts] = earnings_df[facts].replace({'^-$|^None$|^nan$|\)':0, ',':'', '\$':'', ' ':'', '^\(':'-'}, regex=True)
earnings_df[facts] = earnings_df[facts].astype(float)

In [5]:
#convert dimensions from type 'object' to 'string' and remove leading/trailing whitespace
dimensions = ['Name', 'Department', 'Title', 'Postal']
earnings_df[dimensions] = earnings_df[dimensions].astype(str)

#purpose: clean Postal column

#add a 0 to the front of any code with 4 digits
mask = earnings_df['Postal'].str.len() == 4
earnings_df.loc[mask, 'Postal'] = '0' + earnings_df.loc[mask, 'Postal']

#remove delivery route number from any codes that have it (number after hyphen)
earnings_df['Postal'] = earnings_df['Postal'].str.split('-', expand=True)[0]

#any postal codes with non-numeric characters will be set to UNKNOWN
earnings_df.loc[earnings_df['Postal'].str.match('[A-Z]', na=False), 'Postal'] = 'UNKNOWN'

In [6]:
#purpose: clean Name column
earnings_df['Name'] = earnings_df['Name'].replace({'\.':''}, regex=True).str.upper()

In [7]:
#purpose: clean Title column
earnings_df['Title'] = earnings_df['Title'].replace({'\.':'', '-':' ', '(?<=[a-z])([A-Z])':r' \1', '\(':' (', '\)':') ', '\/':' ', '&':' & ', ',':'', '\#':''}, regex=True).str.upper()
earnings_df[dimensions] = earnings_df[dimensions].applymap(lambda x: x.strip())

In [8]:
def abbrev_regex(orig, repl):
    return {'\s'+orig+'\s':' '+repl+' ',
            '\s'+orig+'$':' '+repl,
            '\s'+orig+'\)':' '+repl+')',
            '\s'+orig+'\(':' '+repl+'(', 
            #'\s'+orig+'\#':' '+repl+'#',
            '^'+orig+'\s':repl+' ',
            '^'+orig+'$':repl,
            '\('+orig+'\)':'('+repl+')',
            '\('+orig+'\s':'('+repl+' ',
            '^'+orig+'\(':repl+'('}

abbrevs = {
    'ADMIN':'ADMINISTRATIVE',
    'OFFC':'OFFICER',
    'OFFCR':'OFFICER',
    '\(DET\)':'DETECTIVE',
    'DET':'DETECTIVE',
    'SUPV':'SUPERVISOR',
    'SPV':'SUPERVISOR',
    'EXEC':'EXECUTIVE',
    'ANL':'ANALYST',
    'ANAL':'ANALYST',
    'TECH':'TECHNICIAN',
    'PROJ':'PROJECT',
    'SP PROJ STFF':'SPECIAL PROJECT STAFF',
    'SP PROJECT STFF':'SPECIAL PROJECT STAFF',
    'ACAD':'ACADEMY',
    'INSTR':'INSTRUCTOR',
    'ASST':'ASSISTANT',
    'ASSIST':'ASSISTANT',
    'DEP':'DEPUTY',
    'SUPN':'SUPERINTENDENT',
    'SYS':'SYSTEMS',
    'COOR':'COORDINATOR',
    'COORD':'COORDINATOR',
    'SEC':'SECRETARY',
    'LIEUT':'LIEUTENANT',
    'MAINT':'MAINTENANCE',
    'DIR':'DIRECTOR',
    'MGMT':'MANAGEMENT',
    'MGR':'MANAGER',
    'MNGR':'MANAGER',
    'MANGR':'MANAGER',
    'MED':'MEDICAL',
    'OPER':'OPERATIONS',
    'DATA PROC':'DATA PROCESSING',
    'CORP COUNSEL':'CORPORATION COUNSEL',
    'ASSOC':'ASSOCIATE',
    'COMM SERV':'COMMUNITY SERVICE',
    'COMM':'COMMUNICATIONS',
    'COMMUNIC':'COMMUNICATIONS',
    'COMMUN':'COMMUNICATIONS',
    'BLDG':'BUILDING',
    'SERV':'SERVICE',
    'REG VOTERS':'REGISTRAR OF VOTERS',
    'SVC':'SERVICE',
    'SRV':'SERVICE',
    'EQUIP':'EQUIPMENT',
    'PRIN':'PRINCIPAL',
    'DIST':'DISTRICT',
    'FF':'FIRE FIGHTER',
    'INSTRUC':'INSTRUCTOR',
    'SR':'SENIOR',
    'JR':'JUNIOR',
    'MECH':'MECHANIC',
    'GEN':'GENERAL',
    'ADMN':'ADMINISTRATIVE',
    'ENG':'ENGINEER',
    'STRUCT':'STRUCTURAL',
    'FRPRS':'FOREPERSON',
    'FRPR':'FOREPERSON',
    'CONST':'CONSTRUCTION',
    'LBR':'LABORER',
    'RPR':'REPAIR',
    'REP':'REPAIR',
    'SPEC':'SPECIAL',
    'INCT COMND SP':'INCIDENT COMMAND SPECIALIST',
    'MAS OF F BOAT':'MASTER OF FIRE BOAT',
    'RPPRS':'REPAIR PERSON',
    'REPPRS':'REPAIR PERSON',
    'REPRPRS':'REPAIR PERSON',
    'REPAIRPR':'REPAIR PERSON',
    'REPAIRPRS':'REPAIR PERSON',
    'RPPR':'REPAIR PERSON',
    'WKG':'WORKING',
    'PW':'PUBLIC WORKS',
    'P W':'PUBLIC WORKS',
    'HVY':'HEAVY',
    'MTR':'MOTOR',
    'INSP':'INSPECTOR',
    'INSPEC':'INSPECTOR',
    'TRA':'TRAFFIC',
    'OPR':'OPERATIONS',
    'MEO':'MACHINE EQUIPMENT OPERATOR',
    'CFM':'CERTIFIED FACILITY MANAGER',
    'ELEC EQUIPMENT':'ELECTRIC EQUIPMENT',
    'ACC MANAGEMENT':'ACCOUNT MANAGEMENT',
    'EQUI':'EQUIPMENT',
    'COLL TRS':'COLLECTOR TREASURER',
    'ACNTNG':'ACCOUNTING',
    'CRFTSPRS':'CRAFTSPERSON',
    'COUNSLR':'COUNSELOR',
    'MEMBER BD OF ELECTION':'MEMBER OF BOARD OF ELECTIONS',
    'LIB':'LIBRARIAN',
    'LIBR':'LIBRARIAN',
    'LIBRARIN':'LIBRARIAN',
    'SVCS':'SERVICES',
    'CAMP JO':'CAMP JOY',
    'CAM JO':'CAMP JOY',
    'PROT SER':'PROTECTIVE SERVICES',
    'REL':'RELATIONS'
}

for key in abbrevs:
    earnings_df['Title'] = earnings_df['Title'].replace(abbrev_regex(key, abbrevs[key]), regex=True)
    print(abbrev_regex(key, abbrevs[key]))
    
earnings_df['Title'] = earnings_df['Title'].replace('SPECIAL$', 'SPECIALIST', regex=True)

{'\\sADMIN\\s': ' ADMINISTRATIVE ', '\\sADMIN$': ' ADMINISTRATIVE', '\\sADMIN\\)': ' ADMINISTRATIVE)', '\\sADMIN\\(': ' ADMINISTRATIVE(', '^ADMIN\\s': 'ADMINISTRATIVE ', '^ADMIN$': 'ADMINISTRATIVE', '\\(ADMIN\\)': '(ADMINISTRATIVE)', '\\(ADMIN\\s': '(ADMINISTRATIVE ', '^ADMIN\\(': 'ADMINISTRATIVE('}
{'\\sOFFC\\s': ' OFFICER ', '\\sOFFC$': ' OFFICER', '\\sOFFC\\)': ' OFFICER)', '\\sOFFC\\(': ' OFFICER(', '^OFFC\\s': 'OFFICER ', '^OFFC$': 'OFFICER', '\\(OFFC\\)': '(OFFICER)', '\\(OFFC\\s': '(OFFICER ', '^OFFC\\(': 'OFFICER('}
{'\\sOFFCR\\s': ' OFFICER ', '\\sOFFCR$': ' OFFICER', '\\sOFFCR\\)': ' OFFICER)', '\\sOFFCR\\(': ' OFFICER(', '^OFFCR\\s': 'OFFICER ', '^OFFCR$': 'OFFICER', '\\(OFFCR\\)': '(OFFICER)', '\\(OFFCR\\s': '(OFFICER ', '^OFFCR\\(': 'OFFICER('}
{'\\s\\(DET\\)\\s': ' DETECTIVE ', '\\s\\(DET\\)$': ' DETECTIVE', '\\s\\(DET\\)\\)': ' DETECTIVE)', '\\s\\(DET\\)\\(': ' DETECTIVE(', '^\\(DET\\)\\s': 'DETECTIVE ', '^\\(DET\\)$': 'DETECTIVE', '\\(\\(DET\\)\\)': '(DETECTIVE)', '\\(\

{'\\sOPER\\s': ' OPERATIONS ', '\\sOPER$': ' OPERATIONS', '\\sOPER\\)': ' OPERATIONS)', '\\sOPER\\(': ' OPERATIONS(', '^OPER\\s': 'OPERATIONS ', '^OPER$': 'OPERATIONS', '\\(OPER\\)': '(OPERATIONS)', '\\(OPER\\s': '(OPERATIONS ', '^OPER\\(': 'OPERATIONS('}
{'\\sDATA PROC\\s': ' DATA PROCESSING ', '\\sDATA PROC$': ' DATA PROCESSING', '\\sDATA PROC\\)': ' DATA PROCESSING)', '\\sDATA PROC\\(': ' DATA PROCESSING(', '^DATA PROC\\s': 'DATA PROCESSING ', '^DATA PROC$': 'DATA PROCESSING', '\\(DATA PROC\\)': '(DATA PROCESSING)', '\\(DATA PROC\\s': '(DATA PROCESSING ', '^DATA PROC\\(': 'DATA PROCESSING('}
{'\\sCORP COUNSEL\\s': ' CORPORATION COUNSEL ', '\\sCORP COUNSEL$': ' CORPORATION COUNSEL', '\\sCORP COUNSEL\\)': ' CORPORATION COUNSEL)', '\\sCORP COUNSEL\\(': ' CORPORATION COUNSEL(', '^CORP COUNSEL\\s': 'CORPORATION COUNSEL ', '^CORP COUNSEL$': 'CORPORATION COUNSEL', '\\(CORP COUNSEL\\)': '(CORPORATION COUNSEL)', '\\(CORP COUNSEL\\s': '(CORPORATION COUNSEL ', '^CORP COUNSEL\\(': 'CORPORATION 

{'\\sSPEC\\s': ' SPECIAL ', '\\sSPEC$': ' SPECIAL', '\\sSPEC\\)': ' SPECIAL)', '\\sSPEC\\(': ' SPECIAL(', '^SPEC\\s': 'SPECIAL ', '^SPEC$': 'SPECIAL', '\\(SPEC\\)': '(SPECIAL)', '\\(SPEC\\s': '(SPECIAL ', '^SPEC\\(': 'SPECIAL('}
{'\\sINCT COMND SP\\s': ' INCIDENT COMMAND SPECIALIST ', '\\sINCT COMND SP$': ' INCIDENT COMMAND SPECIALIST', '\\sINCT COMND SP\\)': ' INCIDENT COMMAND SPECIALIST)', '\\sINCT COMND SP\\(': ' INCIDENT COMMAND SPECIALIST(', '^INCT COMND SP\\s': 'INCIDENT COMMAND SPECIALIST ', '^INCT COMND SP$': 'INCIDENT COMMAND SPECIALIST', '\\(INCT COMND SP\\)': '(INCIDENT COMMAND SPECIALIST)', '\\(INCT COMND SP\\s': '(INCIDENT COMMAND SPECIALIST ', '^INCT COMND SP\\(': 'INCIDENT COMMAND SPECIALIST('}
{'\\sMAS OF F BOAT\\s': ' MASTER OF FIRE BOAT ', '\\sMAS OF F BOAT$': ' MASTER OF FIRE BOAT', '\\sMAS OF F BOAT\\)': ' MASTER OF FIRE BOAT)', '\\sMAS OF F BOAT\\(': ' MASTER OF FIRE BOAT(', '^MAS OF F BOAT\\s': 'MASTER OF FIRE BOAT ', '^MAS OF F BOAT$': 'MASTER OF FIRE BOAT', '\\(

{'\\sLIB\\s': ' LIBRARIAN ', '\\sLIB$': ' LIBRARIAN', '\\sLIB\\)': ' LIBRARIAN)', '\\sLIB\\(': ' LIBRARIAN(', '^LIB\\s': 'LIBRARIAN ', '^LIB$': 'LIBRARIAN', '\\(LIB\\)': '(LIBRARIAN)', '\\(LIB\\s': '(LIBRARIAN ', '^LIB\\(': 'LIBRARIAN('}
{'\\sLIBR\\s': ' LIBRARIAN ', '\\sLIBR$': ' LIBRARIAN', '\\sLIBR\\)': ' LIBRARIAN)', '\\sLIBR\\(': ' LIBRARIAN(', '^LIBR\\s': 'LIBRARIAN ', '^LIBR$': 'LIBRARIAN', '\\(LIBR\\)': '(LIBRARIAN)', '\\(LIBR\\s': '(LIBRARIAN ', '^LIBR\\(': 'LIBRARIAN('}
{'\\sLIBRARIN\\s': ' LIBRARIAN ', '\\sLIBRARIN$': ' LIBRARIAN', '\\sLIBRARIN\\)': ' LIBRARIAN)', '\\sLIBRARIN\\(': ' LIBRARIAN(', '^LIBRARIN\\s': 'LIBRARIAN ', '^LIBRARIN$': 'LIBRARIAN', '\\(LIBRARIN\\)': '(LIBRARIAN)', '\\(LIBRARIN\\s': '(LIBRARIAN ', '^LIBRARIN\\(': 'LIBRARIAN('}
{'\\sSVCS\\s': ' SERVICES ', '\\sSVCS$': ' SERVICES', '\\sSVCS\\)': ' SERVICES)', '\\sSVCS\\(': ' SERVICES(', '^SVCS\\s': 'SERVICES ', '^SVCS$': 'SERVICES', '\\(SVCS\\)': '(SERVICES)', '\\(SVCS\\s': '(SERVICES ', '^SVCS\\(': 'SERVIC

In [9]:
earnings_df[['Department', 'Name', 'Title', 'Postal']].head(1000)

,Department,Name,Title,Postal
0,Assessing Department,"ABADI,KIDANI A",PROPERTY OFFICER (ASN),02118
1,ASD Office Of Labor Relation,"ABLON,JORDAN N",ASSISTANT CORPORATION COUNSEL III,02135
2,Transportation-Parking Clerk,"ACCARDI,PATRICIA",CHIEF CLAIMS INVESTIGATOR,02081
3,Boston Public Library,"ACKERLY,LYN E",SPECIAL LIBRARY ASSISTANT I,02118
4,Law Department,"ADAMS,CAREY L",PRINCIPAL CLERK,02131
5,Public Works Department,"ADAMS,DEAN",HIGHWAY MAINTENANCE FOREPERSON (PWD),02124
6,Boston Cntr - Youth & Families,"ADAMS,NATASHA",YOUTH WORKER,02124
7,ASD Human Resources,"ADARIO,ANTHONY J",SUPVISING CLAIMS AGENT (ASD),02132
8,Property Management,"ADDESSA,ROCCO",JUNIOR BUILDING CUSTODIAN,02128
9,Boston Cntr - Youth & Families,"AFONSECA,JOSE",CERTIFIED SEASONAL LIFEGUARD,02124


In [ ]:
#create pivot table displaying the number of people in each dept by year
BPS_str = '^BPS|Elementary|Academy|K-8|Middle|High|School|Acad$|Pilot| EEC$| ELC$| EES$|9-12|Achievement Gap|Student'


#purpose: get pivot table of employee counts by department
    #column indicates what column to count on (i.e. Injured or Total)
    #dept is an optional column that specifies what department to filter on, set to everything by default
def count_by_dept(column, dept=r'(.*?)'):
    
    #create dataframe with department, year, and counts
    dept_counts = earnings_df.loc[earnings_df[column]>0].groupby(['Department', 'Year'])[column].count().reset_index(name="count")
    
    #remove BPS schools from result
    dept_counts = dept_counts.loc[dept_counts['Department'].str.contains(BPS_str, regex=True) == False]
    
    #create the pivot table table, with calculated sums for each row and column
    dept_counts_table = pd.pivot_table(dept_counts, values='count', index='Department', columns='Year', aggfunc='sum', fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return dept_counts_table.iloc[:,0:-1].loc[dept_counts_table['Department'].str.match(dept)]

count_by_dept('Injured')

In [ ]:
#Arts & Cultural Development -> Office of Arts & Culture
#Transportation Department -> Traffic Division
#Women's Commission -> Women's Advancement
#Dept of Voter Mobilization -> Election Division
#Youth Fund -> Youth Engagement & Employment

In [ ]:
#purpose: get pivot table of aggregates by department
    #column indicates what column to use for aggregation (i.e. Injured, Regular, Total, etc.)
    #aggfunc is the aggregation function (mean, sum, max, min)
    #dept is an optional column that specifies what department to filter on, set to everything by default
def agg_by_dept(column, aggfunc, dept=r'(.*?)'):
    
    #create dataframe with department, year, and aggregate column
    dept_costs = earnings_df.loc[earnings_df[column]>0].groupby(['Department', 'Year'])[column].agg(aggfunc).reset_index(name="Costs")
    
    #remove BPS schools from result
    dept_costs = dept_costs.loc[dept_costs['Department'].str.contains(BPS_str, regex=True) == False]
    
    #create the pivot table table, with calculated aggregate for each row and column
    dept_costs_table = pd.pivot_table(dept_costs, values='Costs', index='Department', columns='Year', aggfunc = aggfunc, fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated aggregate by row, doesn't make sense for time series data
    return dept_costs_table.iloc[:,0:-1].loc[dept_costs_table['Department'].str.match(dept)]

agg_by_dept('Injured', 'median', 'Boston Police Department|Boston Fire Department')

In [ ]:
#purpose: get pivot table of employee counts by department and title
    #column indicates what column to count on (i.e. Injured or Total)
    #dept is an optional column that specifies what department to filter on, set to everything by default
    #title is an optional column that specifies what title to filter on, set to everything by default, not reccommended without setting dept first
def count_by_title(column, dept=r'(.*?)', title=r'(.*?)'):
    
    #create dataframe with title, year, and counts
    job_counts = earnings_df.loc[earnings_df['Department'].str.match(dept)]
    job_counts = job_counts.loc[job_counts[column]>0].groupby(['Title', 'Year'])[column].count().reset_index(name="count")
    
    #create the pivot table table, with calculated sums for each row and column
    job_counts_table = pd.pivot_table(job_counts, values='count', index='Title', columns='Year', aggfunc=np.sum, fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return job_counts_table.loc[job_counts_table['Title'].str.match(title)].sort_values(by='All', ascending=False).iloc[:,0:-1]

count_by_title('Injured')

In [ ]:
#purpose: get pivot table of chosen aggregate by department and title
    #column indicates what column to aggregate on (i.e. Injured, Regular, Total, etc.)
    #dept is an optional column that specifies what department to filter on, set to everything by default
    #title is an optional column that specifies what title to filter on, set to everything by default, not reccommended without setting dept first
def agg_by_title(column, aggfunc, dept=r'(.*?)', title=r'(.*?)'):
    
    #create dataframe with title, year, and aggregate column
    job_pay = earnings_df.loc[earnings_df['Department'].str.match(dept)].groupby(['Title', 'Year'])[column].agg(aggfunc).reset_index(name="Costs")
    
    #create the pivot table table, with calculated aggregate for each row and column
    job_pay_table = pd.pivot_table(job_pay, values='Costs', index='Title', columns='Year', fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return job_pay_table.loc[job_pay_table['Title'].str.match(title)].sort_values(by='All', ascending=False).iloc[:,0:-1]

agg_by_title('Total', 'mean', 'Mayor\'s Office', 'Chief Diversity Officer')

In [ ]:
print(earnings_df.loc[(earnings_df['Injured']>0) & (earnings_df['Year']==2014) & (earnings_df['Department']=='Boston Police Department'), ['Name', 'Title', 'Injured']])

In [ ]:
count_by_title('Total', title='Prin')